<a href="https://colab.research.google.com/github/annasvenbro/etudesnordiques/blob/main/20240425_retraceur_dons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Outil pour retracer les dons tels que signalés dans le Sudoc

On cherche à faire un *dataframe* de documents issu des dons au sein d'une bibliothèque donnée à partir de l'interrogation de l'index RPC du Sudoc, avec :
*   le PPN ;
*   le titre ;
*   l'auteur ;
*   les données d'exemplaire ;
*   les données de provenance ;
*   les donateurs / possesseurs ;
*   la cote.

On va donc :
*   interroger l'API SRU du Sudoc ;
*   extraire le contenu des balises XML pertinentes ;
*   présenter le résultat sous forme de *dataframe*.

##Import des paquets nécessaires

In [1]:
import requests as rq
import xml.etree.ElementTree as et
import pandas as pd
import numpy as np
import ipywidgets as widgets
import re
import matplotlib.pyplot as plt
from datetime import datetime

##Création du menu déroulant avec les bibliothèques du réseau Sudoc

In [ ]:
liste_rcr=rq.get("https://www.idref.fr/services/listrcr") #On requête l'URL du webservice.
liste_rcr_text=liste_rcr.text #On voit la tête de la réponse.
lines=liste_rcr_text.split("\n")#Transformation de la réponse en tableau.
header=lines[0].split("\t")
header[0]=header[0].strip("\ufeff") #Pour ne pas avoir de bug dans le dataframe final avec les BOM.
data=[line.split("\t") for line in lines[1:] if line]
df_rcr=pd.DataFrame(data,columns=header) #Transformation en dataframe.
df_rcr.columns=df_rcr.columns.str.strip("\ufeff") #Nettoyage du dataframe (BOM, signe égal, guillemets et autres caractères parasites).
df_rcr["RCR"]=df_rcr["RCR"].str.replace("=","")
df_rcr["RCR"]=df_rcr["RCR"].str.replace('"','')
df_rcr=df_rcr.filter(regex='^RCR$|^LIBELLE$')#Sinon on a un bug à cause des BOM.
df_rcr

In [17]:
rcr_input=None
def nom2rcr(change): #Définition de la fonction pour avoir accès au RCR.
    global rcr_input
    selec_rcr=change["new"]  #Bibliothèque sélectionnée dans le menu déroulant
    rcr_input=df_rcr.loc[df_rcr["LIBELLE"]==selec_rcr,"RCR"].iloc[0]

In [ ]:
texte_autocomplet = widgets.Text(placeholder="Saisir le nom de la bibliothèque") #Création de la fenâtre de saisie pour l'autocomplétion.

liste_bib=widgets.Dropdown(
    options=df_rcr["LIBELLE"],  #Les options du menu déroulant sont les éléments de la colonne "LIBELLE".
    description="Nom de la bibliothèque :"
)
def update_options(change):
    filtered_options = [option for option in df_rcr["LIBELLE"] if texte_autocomplet.value.lower() in option.lower()]
    liste_bib.options = filtered_options

texte_autocomplet.observe(update_options, "value")

display(texte_autocomplet)# Afficher les widgets.
display(liste_bib)

In [ ]:
liste_bib.observe(nom2rcr,names="value") #Sélection de la bibliothèque choisie dans le menu déroulant.
print(rcr_input) #Vérification de l'enreigstrment de la variable (pas très stable).

##Requête de l'API SRU du Sudoc grâce à l'index RPC et avec la limitation du RCR choisi.

### Création de la variable concernant la provenance

In [ ]:
rpc_input=input("Entrez le nom à partir duquel vous voulez rechercher des données de provenance :") #Saisie de la donnée de provenance à analyser.

###Parsage du flux XML obtenu

In [22]:
def get_ppn(rcr_input,rpc_input):
    liste_ppn=[]  # On initialise la liste des PPN.
    start_token=1  # On paramètre le token de départ.
    step=500  # On définit le pas de l'incrément.
    req_nb_notices=rq.get(f"https://www.sudoc.abes.fr/cbs/sru/?operation=searchRetrieve&version=1.1&recordSchema=unimarc&query=rpc%3D{rpc_input}%20and%20rbc%3D{rcr_input}")
    root_nb_notices=et.fromstring(req_nb_notices.content)
    notices = int(root_nb_notices.find('.//{http://www.loc.gov/zing/srw/}numberOfRecords').text) # On requête l'API une première fois pour obtenir le nombre total de notices.
    end_token=((notices-1)//step+1)*step # On paramètre le dernier token pour le tronquer à la 500-aine précédant la valeur totale du nombre d'enregistrements.
    for token in range(start_token, end_token + 1, step): # On fait la requête avec le token pour récupérer les enregistrements.
        req=rq.get(f"https://www.sudoc.abes.fr/cbs/sru/?operation=searchRetrieve&version=1.1&recordSchema=unimarc&maximumRecords=500&startRecord={str(token)}&query=rpc%3D{rpc_input}%20and%20rbc%3D{rcr_input}")
        root_sudoc=et.fromstring(req.content)
        for record in root_sudoc.findall('.//{http://www.loc.gov/zing/srw/}record'):# On trouve la balise du PPN.
          controlfield_003=record.find('.//controlfield[@tag="003"]')
          #if controlfield_003 is not None:
          #  datafield_trad=record.find('.//datafield[@tag="101"]/subfield[@code="c"]') # On sélectionne les PPN qui ont une zone 101$c
          #  if datafield_trad is not None and datafield_trad.text==langue: #On sélectionne les PPN sont la 101$c est égale à celle de la langue choisie.
          liste_ppn.append(controlfield_003.text)
          liste_ppn=[url.replace("http://", "https://") for url in liste_ppn]
    return liste_ppn

In [ ]:
liste_ppn=get_ppn(rcr_input,rpc_input) #Obtention de la liste des PPN concernés par la provenance.
liste_ppn

In [ ]:
len(liste_ppn) #Longueur de la liste des PPN obtenue.

###Création du *dataframe*

On veut, pour une liste de PPN d'un RCR donné, créer un *dataframe* avec les informations concernant chaque document de la liste, avec son PPN (003), son type (008), son titre (200\$a), ses auteurs (200\$f), son éditeur (210\$c ou 214\$c), sa date (210\$c ou 214\$d), ses notes d'exemplaires (316\$a), sa provenance(317\$a), sa cote (930\$a).

####Création de la fonction d'extraction des données

In [25]:
def get_item(ppn):
  req_item=rq.get(f"{ppn}.xml")
  root_item=et.fromstring(req_item.content) #On requête le flux XML associé au PPN.

  if root_item.find(".//controlfield[@tag='008']") is not None: #On extrait le type de document selon la nomenclature de l'Abes.
    controlfield_008=root_item.find(".//controlfield[@tag='008']")
  else: controlfield_008=None
  type_code=controlfield_008.text if controlfield_008 is not None else ""
  if type_code.startswith("Aa"):
    type_doc="Monographie imprimée"
  elif type_code.startswith("Ab"):
    type_doc="Périodique imprimé"
  elif type_code.startswith("Ad"):
    type_doc="Collection imprimée"
  elif type_code.startswith("Ar"):
    type_doc="Recueil factice d'imprimés"
  elif type_code.startswith("Ba"):
    type_doc="Document audiovisuel"
  elif type_code.startswith("Bb"):
    type_doc="Périodique sous forme de documents audiovisuels"
  elif type_code.startswith("Bd"):
    type_doc="Collection de documents audiovisuels"
  elif type_code.startswith("Br"):
    type_doc="Recueil factice de documents audiovisuels"
  elif type_code.startswith("Bs"):
    type_doc="Extrait de document audiovisuel"
  elif type_code.startswith("Fa"):
    type_doc="Manuscrit"
  elif type_code.startswith("Ga"):
    type_doc="Enregistrement sonore musical"
  elif type_code.startswith("Gd"):
    type_doc="Collection d'enregistrements sonores musicaux"
  elif type_code.startswith("Ia"):
    type_doc="Image fixe"
  elif type_code.startswith("Ir"):
    type_doc="Recueil factice d'images fixes"
  elif type_code.startswith("Ka"):
    type_doc="Carte imprimée"
  elif type_code.startswith("Kd"):
    type_doc="Collection de cartes imprimées"
  elif type_code.startswith("Ke"):
    type_doc="Série cartographique"
  elif type_code.startswith("La"):
    type_doc="Partition manuscrite"
  elif type_code.startswith("Ma"):
    type_doc="Partition imprimée"
  elif type_code.startswith("Md"):
    type_doc="Collection de partitions imprimées"
  elif type_code.startswith("Mr"):
    type_doc="Recueil factice de partitions imprimées"
  elif type_code.startswith("Na"):
    type_doc="Enregistrement sonore non musical"
  elif type_code.startswith("Nb"):
    type_doc="Périodique sous forme d'enregistrements sonores non musicaux"
  elif type_code.startswith("Nd"):
    type_doc="Collection d'enregistrements sonores non musicaux"
  elif type_code.startswith("Oa"):
    type_doc="Monographie électronique"
  elif type_code.startswith("Ob"):
    type_doc="Périodique électronique"
  elif type_code.startswith("Od"):
    type_doc="Collection de documents électroniques"
  elif type_code.startswith("Or"):
    type_doc="Recueil factice de documents électroniques"
  elif type_code.startswith("Os"):
    type_doc="	Partie de document électronique"
  elif type_code.startswith("Pa"):
    type_doc="	Carte manuscrite"
  elif type_code.startswith("Va"):
    type_doc="Objet"
  elif type_code.startswith("Za"):
    type_doc="Document multimédia multisupport"
  elif type_code.startswith("Zb"):
    type_doc="Périodique multimédia multisupport"
  elif type_code.startswith("Zd"):
    type_doc="Collection de documents multimédias multisupports"
  elif type_code.startswith("Zr"):
    type_doc="Recueil factice de documents multimédias multisupports"
  else :
    type_doc="Type non renseigné"

  titre=root_item.find('.//datafield[@tag="200"]/subfield[@code="a"]') #On extrait le titre.
  titre=titre.text if titre is not None else "Titre non renseigné"

  auteur=root_item.find('.//datafield[@tag="200"]/subfield[@code="f"]') #On extrait l'auteur.
  auteur=auteur.text if auteur is not None else "Auteur non renseigné"

  if root_item.find('.//datafield[@tag="210"]/subfield[@code="c"]') is not None: #On extrait l'éditeur.
    editeur=root_item.find('.//datafield[@tag="210"]/subfield[@code="c"]')
  elif root_item.find('.//datafield[@tag="214"]/subfield[@code="c"]') is not None:
    editeur=root_item.find('.//datafield[@tag="214"]/subfield[@code="c"]')
  else: editeur=None
  editeur=editeur.text if editeur is not None else "Éditeur non renseigné"

  if root_item.find('.//datafield[@tag="210"]/subfield[@code="d"]') is not None: #On extrait la date.
    date=root_item.find('.//datafield[@tag="210"]/subfield[@code="d"]')
  elif root_item.find('.//datafield[@tag="214"]/subfield[@code="d"]') is not None:
    date=root_item.find('.//datafield[@tag="214"]/subfield[@code="d"]')
  else: date=None
  date=date.text if date is not None else "Date non renseignée"
  if date:
    match=re.search(r"\d{4}", date)
    if match:
      date= match.group()
  else:
    date="Date non renseignée"

  exemplaire=[]
  datafields_316=root_item.findall('.//datafield[@tag="316"]') #On extrait les données d'exemplaire spécifiques au RCR choisi.
  for datafield_316 in datafields_316:
    subfield_5=datafield_316.find('subfield[@code="5"]')
    subfield_a=datafield_316.find('subfield[@code="a"]')
    if subfield_5 is not None and subfield_a is not None:
      if subfield_5.text.startswith(rcr_input):
        exemplaire.append(subfield_a.text.strip())
  if not exemplaire :
    exemplaire.append("Données d'exemplaire non renseignées")

  provenance=[]
  datafields_317=root_item.findall('.//datafield[@tag="317"]') #On extrait les données de provenance spécifiques au RCR choisi.
  for datafield_317 in datafields_317:
    subfield_5=datafield_317.find('subfield[@code="5"]')
    subfield_a=datafield_317.find('subfield[@code="a"]')
    if subfield_5 is not None and subfield_a is not None:
      if subfield_5.text.startswith(rcr_input):
        provenance.append(subfield_a.text.strip())
  if not provenance :
    provenance.append("Provenance non renseignée")

  donateur_possesseur=[]
  tags_don= ["702","712","722"]
  code_don=["320","390"]
  for tag in tags_don:
    for datafield in root_item.findall(f".//datafield[@tag='{tag}']"):
    #On vérifie si la sous-balise <subfield code="4">320/390</subfield> pour le(s) anciens possesseur(s) / donateur(s) existe.
      subfields_4=datafield.findall('subfield[@code="4"]')
      if any(subfield_4.text=="320" for subfield_4 in subfields_4) or any(subfield_4.text=="390" for subfield_4 in subfields_4):
        #On récupère les sous-balises d'identification pour le(s) ancien(s) possesseurs(s) / donateur(s).
        subfield_a=datafield.find('subfield[@code="a"]')
        subfield_b=datafield.find('subfield[@code="b"]')
        if subfield_a is not None and subfield_b is not None:
            #On ajoute cette identification à la liste des résultats
            donateur_possesseur.append((subfield_a.text,subfield_b.text))
  if not donateur_possesseur:
    donateur_possesseur.append("Pas de donateur / possesseur renseigné")

  cote=[]
  cote_tags=root_item.findall('.//datafield[@tag="930"]/subfield[@code="b"][.="{}"]/..'.format(rcr_input)) #On extrait les cotes spécifiques au RCR choisi.
  for cote_tag in cote_tags :
    subfields_a = cote_tag.findall('./subfield[@code="a"]')
    for subfield_a in subfields_a:
        cote.append(subfield_a.text)

  autres_rcr=[]
  datafields_930=root_item.findall('.//datafield[@tag="930"]') #On extrait les autres RCR où le document est présent.
  for datafield_930 in datafields_930:
    subfield_b=datafield_930.find('subfield[@code="b"]')
    if subfield_b is not None:
      rcr_value=subfield_b.text
      if rcr_value!=rcr_input:
        autres_rcr.append(rcr_value)


  return type_doc,titre,auteur,editeur,date,exemplaire,provenance,donateur_possesseur,cote,autres_rcr

####Application à la liste de PPN pour obtenir le *dataframe* détaillé

In [ ]:
data={"PPN": [],"Type de document": [],"Titre": [],"Auteurs": [],"Éditeur": [],"Date": [],"Données d'exemplaire":[],"Provenance": [], "Donateur / possesseur":[],"Cote": [], "Autres localisations Sudoc":[]}
for ppn in liste_ppn:
        item_info=get_item(ppn)
        data["PPN"].append(ppn)
        data["Type de document"].append(item_info[0])
        data["Titre"].append(item_info[1])
        data["Auteurs"].append(item_info[2])
        data["Éditeur"].append(item_info[3])
        data["Date"].append(item_info[4])
        data["Données d'exemplaire"].append(item_info[5])
        data["Provenance"].append(item_info[6])
        data["Donateur / possesseur"].append(item_info[7])
        data["Cote"].append(item_info[8])
        data["Autres localisations Sudoc"].append(item_info[9])
df_don=pd.DataFrame(data)
df_don

In [ ]:
rcr_autres=df_don["Autres localisations Sudoc"].explode() #On convertit la liste des numéros des autres RCR où le document est présent en liste des noms de RCR.
rcr_autres=rcr_autres.drop_duplicates()
rcr_autres=rcr_autres.dropna()
rcr_autres=rcr_autres.to_list()
liste_rcr_autres=[]
for rcr in rcr_autres:
  rows=df_rcr[df_rcr["RCR"]==rcr]
  if not rows.empty:
        liste_rcr_autres.extend(rows["LIBELLE"].tolist())
liste_rcr_autres.sort()
liste_rcr_autres

In [ ]:
len(liste_rcr_autres) #On voit dans combien d'autres RCR il y a des documents du fonds analysé.

In [ ]:
nb_unica=len(df_don[df_don["Autres localisations Sudoc"].apply(lambda x:len(x)==0)]) #On compte le nombre d'unica dans le Sudoc présents dans le fonds analysé (en comptant le nombre d'éléments pour lesquels il n'y a pas d'autres localisations dans le Sudoc).
print(nb_unica)
pourcentage_unica=round((len(df_don[df_don["Autres localisations Sudoc"].apply(lambda x:len(x)==0)])/len(liste_ppn)) * 100,2) #On calcule le pourcentage d'unica.
pourcentage_unica

####Export historicisé du *dataframe*

In [ ]:
aujourdhui=datetime.now().strftime("%Y%m%d") #On exporte les données du fonds analysé en csv historicisé.
nom_fichier=f"{aujourdhui}_export_notices_don_{rpc_input}_RCR{rcr_input}.csv"
df_don.to_csv(nom_fichier,index=True,encoding="utf-8")